In [1]:
import pandas as pd 
import xarray as xr

## Energy data

In [2]:
energy_data_20200920_20240118 = pd.read_csv("HEFTcom24/data/energy_data/Energy_Data_20200920_20240118.csv")
energy_data_20240119_20240519 = pd.read_csv("HEFTcom24/data/energy_data/energy_data_20240119_20240519.csv")

In [3]:
energy_data_old = energy_data_20200920_20240118.copy()
energy_data_new = energy_data_20240119_20240519.copy()

energy_data_old["dtm"] = pd.to_datetime(energy_data_old["dtm"])
energy_data_old["Wind_MWh_credit"] = 0.5 * energy_data_old["Wind_MW"] - energy_data_old["boa_MWh"]
energy_data_old["Solar_MWh_credit"] = 0.5 * energy_data_old["Solar_MW"]

energy_data_new["dtm"] = pd.to_datetime(energy_data_new["dtm"])
energy_data_new["Wind_MWh_credit"] = 0.5 * energy_data_new["Wind_MW"] - energy_data_new["boa_MWh"]
energy_data_new["Solar_MWh_credit"] = 0.5 * energy_data_new["Solar_MW"]

energy_data_merged = pd.concat([energy_data_old, energy_data_new], ignore_index=True)

In [4]:
energy_data = energy_data_merged.copy()

In [89]:
energy_data

,dtm,MIP,Solar_MW,Solar_capacity_mwp,Solar_installedcapacity_mwp,Wind_MW,SS_Price,boa_MWh,DA_Price,Wind_MWh_credit,Solar_MWh_credit
0,2020-09-20 00:00:00+00:00,20.06,0.0,2130.537493,2228.208777,996.284,2.50000,0.0,32.17,498.142,0.0
1,2020-09-20 00:30:00+00:00,19.77,0.0,2130.537493,2228.208777,957.576,15.00000,0.0,32.17,478.788,0.0
2,2020-09-20 01:00:00+00:00,28.68,0.0,2130.537493,2228.208777,941.044,47.95000,0.0,32.00,470.522,0.0
3,2020-09-20 01:30:00+00:00,28.97,0.0,2130.537493,2228.208777,964.366,29.13000,0.0,32.00,482.183,0.0
4,2020-09-20 02:00:00+00:00,28.19,0.0,2130.537493,2228.208777,918.432,28.95000,0.0,31.99,459.216,0.0
...,...,...,...,...,...,...,...,...,...,...,...
64219,2024-05-19 21:30:00+00:00,77.40,0.0,2573.542092,2741.831037,227.518,56.20000,0.0,76.81,113.759,0.0
64220,2024-05-19 22:00:00+00:00,80.82,0.0,2573.542092,2741.831037,197.090,102.00023,0.0,71.11,98.545,0.0
64221,2024-05-19 22:30:00+00:00,75.69,0.0,2573.542092,2741.831037,171.306,99.00000,0.0,71.11,85.653,0.0
64222,2024-05-19 23:00:00+00:00,78.98,0.0,2573.542092,2741.831037,163.164,106.44988,0.0,66.51,81.582,0.0


## Weather forecasts

In [5]:
import os
import pandas as pd
import xarray as xr

def process_and_concat_files(directory):
    dfs = []

    for file in os.listdir(directory):
        if file.endswith(".nc"):
            file_path = os.path.join(directory, file)
            ds = xr.open_dataset(file_path)
            df = ds.to_dataframe().reset_index()
            
            if "ref_datetime" in df.columns and "valid_datetime" in df.columns:
                df.rename(columns={"ref_datetime": "reference_time", "valid_datetime": "valid_time"}, inplace=True)
            elif "reference_time" not in df.columns or "valid_time" not in df.columns:
                raise ValueError("Neither 'ref_datetime' and 'valid_datetime' nor 'reference_time' and 'valid_time' found in the dataset.")
            
            df["reference_time"] = df["reference_time"].dt.tz_localize("UTC")
            df["valid_time"] = df["reference_time"] + pd.to_timedelta(df["valid_time"], unit="hours")
            df["reference_time"] = df["reference_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            df["valid_time"] = df["valid_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            dfs.append(df)
    
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df["reference_time"] = pd.to_datetime(concatenated_df["reference_time"])
    concatenated_df["valid_time"] = pd.to_datetime(concatenated_df["valid_time"])
    concatenated_df["reference_time"] = concatenated_df["reference_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    concatenated_df["valid_time"] = concatenated_df["valid_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    
    return concatenated_df

In [6]:
base_dir = "HEFTcom24/data"

dwd_demand_df = process_and_concat_files(os.path.join(base_dir, "dwd_demand"))
dwd_pes10_df = process_and_concat_files(os.path.join(base_dir, "dwd_pes10"))
dwd_hornsea_1_df = process_and_concat_files(os.path.join(base_dir, "dwd_hornsea_1"))

ncep_demand_df = process_and_concat_files(os.path.join(base_dir, "ncep_demand"))
ncep_pes10_df = process_and_concat_files(os.path.join(base_dir, "ncep_pes10"))
ncep_hornsea_1_df = process_and_concat_files(os.path.join(base_dir, "ncep_hornsea_1"))

## Mean Percentage Diff

In [7]:
dwd_test = dwd_hornsea_1_df.groupby(["reference_time", "valid_time", "longitude", "latitude"]).mean().reset_index()

In [8]:
dwd_test1 = dwd_test.copy()

In [24]:
for i in range(1, 36):
    dwd_test1[f"perc_diff_{i}"] = (dwd_test1["WindSpeed:100"] - dwd_test1["WindSpeed:100"].shift(-i))# / dwd_test1["WindSpeed:100"] * 100

In [31]:
dwd_test1[dwd_test1["perc_diff_2"] > 6] 

,reference_time,valid_time,longitude,latitude,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,...,perc_diff_26,perc_diff_27,perc_diff_28,perc_diff_29,perc_diff_30,perc_diff_31,perc_diff_32,perc_diff_33,perc_diff_34,perc_diff_35
3130,2020-09-20 00:00:00,2020-09-24 06:00:00,2.027,54.03,78.325760,12.230774,249.352356,249.625259,14.446177,16.669464,...,8.454851,7.892223,7.569984,7.915836,7.915836,7.090984,8.352722,7.939036,7.541823,7.662777
3131,2020-09-20 00:00:00,2020-09-24 06:00:00,2.027,54.10,73.458572,12.554016,255.263550,255.727737,14.931273,17.539986,...,8.762745,8.440506,8.786358,8.786358,7.961506,9.223244,8.809558,8.412345,8.533298,8.533298
9331,2020-09-20 12:00:00,2020-09-23 13:00:00,1.767,53.84,94.978195,15.205566,209.058807,211.456253,6.136590,8.575730,...,2.217823,2.217823,5.753573,3.099650,5.806690,6.348791,4.942386,4.942386,5.131303,2.162550
9421,2020-09-20 12:00:00,2020-09-23 15:00:00,1.962,53.84,94.489174,15.007751,225.962173,225.839127,6.280030,7.684906,...,4.476149,4.476149,4.042365,3.770056,5.173798,6.085272,5.039148,5.039148,3.810954,2.627956
12425,2020-09-20 18:00:00,2020-09-23 12:00:00,1.702,54.10,92.196167,13.860901,21.759888,22.890015,7.447649,8.157602,...,2.516127,5.046050,4.683619,4.683619,2.936748,7.211439,5.480062,2.347166,-1.161321,-1.161321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17640610,2024-05-15 12:00:00,2024-05-20 12:00:00,2.027,54.03,77.687225,10.420715,20.580292,21.015106,7.731364,8.948258,...,8.373289,8.031327,7.739954,7.368941,7.368941,7.285115,8.171923,7.853416,7.590491,7.266784
17640611,2024-05-15 12:00:00,2024-05-20 12:00:00,2.027,54.10,74.380585,10.451965,20.786011,20.982086,8.161761,9.196252,...,8.279321,7.987947,7.616935,7.616935,7.533108,8.419916,8.101409,7.838485,7.514778,7.514778
17674090,2024-05-18 00:00:00,2024-05-23 00:00:00,2.027,54.03,96.540459,12.282776,306.477051,312.259918,9.496290,13.164660,...,7.687050,7.385567,7.127760,6.354115,6.354115,6.063227,7.658544,7.366518,7.017605,6.242725
17674091,2024-05-18 00:00:00,2024-05-23 00:00:00,2.027,54.10,96.270927,12.409729,309.585541,314.219299,10.293956,13.601790,...,7.822697,7.564890,6.791245,6.791245,6.500357,8.095674,7.803648,7.454735,6.679855,6.679855


## HORNSEA_1

In [68]:
dwd_hornsea_1_df.head()

,reference_time,valid_time,latitude,longitude,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100
0,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.702,59.105434,4.916626,54.525970,54.250305,6.547671,6.854635
1,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.767,58.628872,4.860962,54.288940,54.106964,6.524992,6.817597
2,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.832,58.365200,4.823853,53.520721,53.448120,6.493145,6.795032
3,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.897,58.441372,4.814087,52.447235,52.500397,6.495604,6.822487
4,2024-01-08 00:00:00,2024-01-08 00:00:00,53.77,1.962,58.501919,4.804321,51.579865,51.684021,6.521748,6.856290


In [71]:
print(dwd_hornsea_1_df["RelativeHumidity"].isna().sum())
print(dwd_hornsea_1_df["Temperature"].isna().sum())
print(dwd_hornsea_1_df["WindDirection"].isna().sum())
print(dwd_hornsea_1_df["WindDirection:100"].isna().sum())
print(dwd_hornsea_1_df["WindSpeed"].isna().sum())
print(dwd_hornsea_1_df["WindSpeed:100"].isna().sum())

print(ncep_hornsea_1_df["RelativeHumidity"].isna().sum())
print(ncep_hornsea_1_df["Temperature"].isna().sum())
print(ncep_hornsea_1_df["WindDirection"].isna().sum())
print(ncep_hornsea_1_df["WindDirection:100"].isna().sum())
print(ncep_hornsea_1_df["WindSpeed"].isna().sum())
print(ncep_hornsea_1_df["WindSpeed:100"].isna().sum())

13356
13392
13356
12204
13356
12204
3456
3627
5733
5733
5733
5733


In [72]:
dwd_hornsea_1_df.dropna(subset=["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"], inplace=True)
ncep_hornsea_1_df.dropna(subset=["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"], inplace=True)

In [32]:
dwd_h1df_mean = dwd_hornsea_1_df.groupby(["reference_time", "valid_time"])[["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"]].mean().reset_index()
ncep_h1df_mean = ncep_hornsea_1_df.groupby(["reference_time", "valid_time"])[["RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"]].mean().reset_index()

In [33]:
h1df_conc = dwd_h1df_mean.merge(ncep_h1df_mean, on=["reference_time", "valid_time"], suffixes=("_dwd", "_ncep"), how= "outer")
# h1df_conc["RelativeHumidity"] = h1df_conc[["RelativeHumidity_dwd", "RelativeHumidity_ncep"]].mean(axis=1)
# h1df_conc["Temperature"] = h1df_conc[["Temperature_dwd", "Temperature_ncep"]].mean(axis=1)
# h1df_conc["WindDirection"] = h1df_conc[["WindDirection_dwd", "WindDirection_ncep"]].mean(axis=1)
# h1df_conc["WindDirection:100"] = h1df_conc[["WindDirection:100_dwd", "WindDirection:100_ncep"]].mean(axis=1)
# h1df_conc["WindSpeed"] = h1df_conc[["WindSpeed_dwd", "WindSpeed_ncep"]].mean(axis=1)
# h1df_conc["WindSpeed:100"] = h1df_conc[["WindSpeed:100_dwd", "WindSpeed:100_ncep"]].mean(axis=1)
h1df_conc["reference_time"] = pd.to_datetime(h1df_conc["reference_time"], format="%Y-%m-%d %H:%M:%S")
h1df_conc["valid_time"] = pd.to_datetime(h1df_conc["valid_time"], format="%Y-%m-%d %H:%M:%S")
#h1df_conc = h1df_conc[["reference_time", "valid_time", "RelativeHumidity", "Temperature", "WindDirection", "WindDirection:100", "WindSpeed", "WindSpeed:100"]]

In [34]:
h1df_conc

,reference_time,valid_time,RelativeHumidity_dwd,Temperature_dwd,WindDirection_dwd,WindDirection:100_dwd,WindSpeed_dwd,WindSpeed:100_dwd,RelativeHumidity_ncep,Temperature_ncep,WindDirection_ncep,WindDirection:100_ncep,WindSpeed_ncep,WindSpeed:100_ncep
0,2020-09-20,2020-09-20 00:00:00,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,84.066673,15.450012,56.139633,58.721077,9.116803,11.338992
1,2020-09-20,2020-09-20 01:00:00,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,84.800003,15.344140,56.170006,58.268215,9.513059,11.693331
2,2020-09-20,2020-09-20 02:00:00,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223,85.066666,15.267812,59.059029,60.833061,9.662121,11.740044
3,2020-09-20,2020-09-20 03:00:00,82.977676,15.459883,56.277557,57.054367,9.455199,11.135883,85.177780,15.199863,59.058838,60.966099,9.540911,11.492398
4,2020-09-20,2020-09-20 04:00:00,82.238251,15.506005,54.625362,55.247120,9.515403,11.254492,85.588890,15.135688,58.351414,60.182644,9.456993,11.297719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114015,2024-05-19,2024-06-03 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,89.977776,13.626984,75.030380,83.595985,3.930665,5.688190
1114016,2024-05-19,2024-06-03 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,88.566666,14.016028,66.449463,72.859047,5.794817,8.217826
1114017,2024-05-19,2024-06-03 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,89.833336,13.756535,66.883980,73.883461,7.639992,11.537951
1114018,2024-05-19,2024-06-03 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,94.166664,13.350006,92.814087,95.731903,7.890777,11.280699


In [75]:
h1df_conc.set_index("valid_time").groupby("reference_time").resample("30min").interpolate("linear")

reference_time  RelativeHumidity  \
reference_time valid_time                                             
2020-09-20     2020-09-20 00:00:00     2020-09-20         84.640213   
               2020-09-20 00:30:00     2020-09-20         84.722801   
               2020-09-20 01:00:00     2020-09-20         84.805389   
               2020-09-20 01:30:00     2020-09-20         84.645615   
               2020-09-20 02:00:00     2020-09-20         84.485832   
...                                           ...               ...   
2024-05-19     2024-06-03 22:00:00     2024-05-19         94.396294   
               2024-06-03 22:30:00     2024-05-19         94.511108   
               2024-06-03 23:00:00     2024-05-19         94.625931   
               2024-06-03 23:30:00     2024-05-19         94.740746   
               2024-06-04 00:00:00     2024-05-19         94.855560   

                                    Temperature  WindDirection  \
reference_time valid_time                                        
2020-09-20     2020-09-20 00:00:00    15.433341      58.863857   
               2020-09-20 00:30:00    15.404793      58.679245   
               2020-09-20 01:00:00    15.376245      58.494629   
               2020-09-20 01:30:00    15.378597      59.062771   
               2020-09-20 02:00:00    15.380949      59.630913   
...                                         ...            ...   
2024-05-19     2024-06-03 22:00:00    13.166626     117.014015   
               2024-06-03 22:30:00    13.074936     129.113983   
               2024-06-03 23:00:00    12.983246     141.213943   
               2024-06-03 23:30:00    12.891556     153.313904   
               2024-06-04 00:00:00    12.799866     165.413864   

                                    WindDirection:100  WindSpeed  \
reference_time valid_time                                          
2020-09-20     2020-09-20 00:00:00          60.403130   9.580215   
               2020-09-20 00:30:00          60.110813   9.610234   
               2020-09-20 01:00:00          59.818497   9.640253   
               2020-09-20 01:30:00          60.330841   9.609314   
               2020-09-20 02:00:00          60.843185   9.578375   
...                                               ...        ...   
2024-05-19     2024-06-03 22:00:00         122.903908   7.484355   
               2024-06-03 22:30:00         136.489899   7.281144   
               2024-06-03 23:00:00         150.075912   7.077933   
               2024-06-03 23:30:00         163.661911   6.874722   
               2024-06-04 00:00:00         177.247910   6.671511   

                                    WindSpeed:100  
reference_time valid_time                          
2020-09-20     2020-09-20 00:00:00      11.570798  
               2020-09-20 00:30:00      11.582490  
               2020-09-20 01:00:00      11.594182  
               2020-09-20 01:30:00      11.535408  
               2020-09-20 02:00:00      11.476633  
...                                           ...  
2024-05-19     2024-06-03 22:00:00      10.662412  
               2024-06-03 22:30:00      10.353268  
               2024-06-03 23:00:00      10.044124  
               2024-06-03 23:30:00       9.734981  
               2024-06-04 00:00:00       9.425837  

[4095716 rows x 7 columns]

## PES10

In [66]:
dwd_pes10_df.head(10)

,reference_time,valid_time,point,CloudCover,SolarDownwardRadiation,Temperature,latitude,longitude
0,2024-01-08 00:00:00,2024-01-08 00:00:00,0,0.409668,0.0,0.316040,52.487256,0.401245
1,2024-01-08 00:00:00,2024-01-08 00:00:00,1,0.508945,0.0,1.136353,52.877668,0.790653
2,2024-01-08 00:00:00,2024-01-08 00:00:00,2,0.546328,0.0,0.637329,52.135428,-0.264034
3,2024-01-08 00:00:00,2024-01-08 00:00:00,3,0.302520,0.0,0.057251,52.488050,-0.126705
4,2024-01-08 00:00:00,2024-01-08 00:00:00,4,0.621309,0.0,1.159790,51.956370,0.658817
5,2024-01-08 00:00:00,2024-01-08 00:00:00,5,0.492266,0.0,2.122681,52.249918,1.389408
6,2024-01-08 00:00:00,2024-01-08 00:00:00,6,0.637207,0.0,2.217407,52.641648,1.350956
7,2024-01-08 00:00:00,2024-01-08 00:00:00,7,0.744160,0.0,1.062134,52.270091,0.708256
8,2024-01-08 00:00:00,2024-01-08 00:00:00,8,0.734180,0.0,0.793579,52.196077,0.153446
9,2024-01-08 00:00:00,2024-01-08 00:00:00,9,0.394082,0.0,0.465454,52.708262,0.730228


In [78]:
print(dwd_pes10_df["SolarDownwardRadiation"].isna().sum())
print(dwd_pes10_df["Temperature"].isna().sum())
print(dwd_pes10_df["CloudCover"].isna().sum())

print(ncep_pes10_df["SolarDownwardRadiation"].isna().sum())
print(ncep_pes10_df["Temperature"].isna().sum())
print(ncep_pes10_df["CloudCover"].isna().sum())

0
0
0
104180
10173
15313


In [79]:
dwd_pes10_df = dwd_pes10_df.dropna(subset= ["Temperature", "SolarDownwardRadiation", "CloudCover"])
ncep_pes10_df = ncep_pes10_df.dropna(subset= ["Temperature", "SolarDownwardRadiation", "CloudCover"])

In [81]:
pivot_dwd_pes10_df = dwd_pes10_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "CloudCover", "SolarDownwardRadiation"]
)
pivot_dwd_pes10_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_dwd_pes10_df.columns.values]
pivot_dwd_pes10_df.reset_index(inplace=True)

pivot_ncep_pes10_df = ncep_pes10_df.pivot_table(
    index=["valid_time", "reference_time"],
    columns="point",
    values=["Temperature", "CloudCover", "SolarDownwardRadiation"]
)
pivot_ncep_pes10_df.columns = ['_Point'.join(map(str, col)).strip() for col in pivot_ncep_pes10_df.columns.values]
pivot_ncep_pes10_df.reset_index(inplace=True)


In [82]:
pivot_dwd_pes10_df["Mean_CloudCover"] = pivot_dwd_pes10_df.filter(like="CloudCover").mean(axis=1)
pivot_dwd_pes10_df["Mean_Temperature"] = pivot_dwd_pes10_df.filter(like="Temperature").mean(axis=1)
pivot_dwd_pes10_df["Mean_SolarDownwardRadiation"] = pivot_dwd_pes10_df.filter(like="SolarDownwardRadiation").mean(axis=1)

pivot_dwd_pes10_df["Std_CloudCover"] = pivot_dwd_pes10_df.filter(like="CloudCover").std(axis=1)
pivot_dwd_pes10_df["Std_Temperature"] = pivot_dwd_pes10_df.filter(like="Temperature").std(axis=1)
pivot_dwd_pes10_df["Std_SolarDownwardRadiation"] = pivot_dwd_pes10_df.filter(like="SolarDownwardRadiation").std(axis=1)

pivot_dwd_pes10_df["reference_time"] = pd.to_datetime(pivot_dwd_pes10_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_dwd_pes10_df["valid_time"] = pd.to_datetime(pivot_dwd_pes10_df["valid_time"], format="%Y-%m-%d %H:%M:%S")


pivot_ncep_pes10_df["Mean_CloudCover"] = pivot_ncep_pes10_df.filter(like="CloudCover").mean(axis=1)
pivot_ncep_pes10_df["Mean_Temperature"] = pivot_ncep_pes10_df.filter(like="Temperature").mean(axis=1)
pivot_ncep_pes10_df["Mean_SolarDownwardRadiation"] = pivot_ncep_pes10_df.filter(like="SolarDownwardRadiation").mean(axis=1)

pivot_ncep_pes10_df["Std_CloudCover"] = pivot_ncep_pes10_df.filter(like="CloudCover").std(axis=1)
pivot_ncep_pes10_df["Std_Temperature"] = pivot_ncep_pes10_df.filter(like="Temperature").std(axis=1)
pivot_ncep_pes10_df["Std_SolarDownwardRadiation"] = pivot_ncep_pes10_df.filter(like="SolarDownwardRadiation").std(axis=1)

pivot_ncep_pes10_df["reference_time"] = pd.to_datetime(pivot_ncep_pes10_df["reference_time"], format="%Y-%m-%d %H:%M:%S")
pivot_ncep_pes10_df["valid_time"] = pd.to_datetime(pivot_ncep_pes10_df["valid_time"], format="%Y-%m-%d %H:%M:%S")

In [88]:
pivot_ncep_pes10_df

,valid_time,reference_time,CloudCover_Point0,CloudCover_Point1,CloudCover_Point2,CloudCover_Point3,CloudCover_Point4,CloudCover_Point5,CloudCover_Point6,CloudCover_Point7,...,Temperature_Point16,Temperature_Point17,Temperature_Point18,Temperature_Point19,Mean_CloudCover,Mean_Temperature,Mean_SolarDownwardRadiation,Std_CloudCover,Std_Temperature,Std_SolarDownwardRadiation
0,2020-09-20 00:00:00,2020-09-20 00:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,13.150024,14.950012,16.050018,12.750000,0.02600,13.940016,0.000000,0.113331,1.095399,0.000000
1,2020-09-20 01:00:00,2020-09-20 00:00:00,0.000,0.300,0.000,0.000,0.000,0.000,0.250,0.000,...,13.004150,14.864136,15.964142,12.634155,0.05200,13.853142,0.000000,0.126436,1.105464,0.000000
2,2020-09-20 02:00:00,2020-09-20 00:00:00,0.000,0.140,0.000,0.000,0.000,0.000,0.090,0.000,...,13.115601,14.815582,15.965576,12.575592,0.02500,13.841084,0.000000,0.054818,1.027557,0.000000
3,2020-09-20 03:00:00,2020-09-20 00:00:00,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.010,...,12.980988,14.790985,15.920990,12.470978,0.01350,13.770979,0.000000,0.027069,0.990799,0.000000
4,2020-09-20 04:00:00,2020-09-20 00:00:00,0.210,0.000,0.000,0.000,0.380,0.000,0.000,0.220,...,12.713470,14.673462,15.873444,12.363464,0.09750,13.648458,0.000000,0.122551,0.999966,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105283,2024-06-03 15:00:00,2024-05-19 00:00:00,0.366,0.164,0.999,0.820,1.000,0.994,0.654,0.883,...,21.193817,18.293793,13.993805,20.093811,0.66180,19.013805,670.674002,0.313400,1.821702,11.026827
1105284,2024-06-03 18:00:00,2024-05-18 18:00:00,0.086,0.041,0.049,0.050,0.050,0.050,0.050,0.067,...,16.704926,14.104919,12.204926,17.504913,0.12255,15.259916,210.668999,0.153571,1.381836,58.794126
1105285,2024-06-03 18:00:00,2024-05-19 00:00:00,0.951,1.000,1.000,1.000,0.986,1.000,1.000,1.000,...,16.334320,14.634308,13.934296,13.234314,0.97700,14.979307,47.321000,0.054375,1.085574,48.393539
1105286,2024-06-03 21:00:00,2024-05-19 00:00:00,0.006,0.969,0.226,0.420,0.119,0.139,0.015,0.151,...,13.550018,12.550018,13.649994,12.550018,0.20220,12.945009,0.000000,0.279000,0.585216,0.000000


In [91]:
import plotly.express as px

px.line(pivot_ncep_pes10_df[pivot_ncep_pes10_df["valid_time"] > "2024-03-01"], x="valid_time", y="Std_SolarDownwardRadiation")